# VisionARM - Hand Gesture Controlled Robotic Arm System
## University of Florida | Artificial Intelligence System Project

This notebook contains our main VisionARM system with:
- Real-time hand gesture detection using MediaPipe
- SVM classification for gesture recognition
- User authentication and feedback system for our UI
- Arduino Integration for deployment of our system
- Performance monitoring with Prometheus
- Gradio web interface

---
## 1. Installation & Dependencies

In [1]:
!pip install gradio
!pip install pyserial
!pip install prometheus-client psutil
!pip install plotly

---
## 2. Import Libraries

In [2]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Computer Vision
import mediapipe as mp
import cv2

# Machine Learning
import joblib

# System & Threading
import time
import threading
import serial

# Web Interface
import gradio as gr

# User Management
from user_manager import get_user_manager

# Monitoring & Metrics
from prometheus_client import Counter, Histogram, Gauge, start_http_server, Summary
import psutil
from collections import deque
import plotly.graph_objs as go
from datetime import datetime

pandas already installed


---
**All the necessary Emojis for this project has been taken from the website-**
[emojipedia](https://emojipedia.org/smileys)
---

---
## 3. Prometheus Metrics Setup

In [3]:
# Clearing any existing metrics (if any)
from prometheus_client import REGISTRY
collectors = list(REGISTRY._collector_to_names.keys())
for collector in collectors:
    try:
        REGISTRY.unregister(collector)
    except Exception:
        pass

In [4]:
# Creating Prometheus Metrics
latency_histogram = Histogram('gesture_detection_latency_seconds', 
                              'Latency of gesture detection',
                              buckets=[0.01, 0.02, 0.03, 0.05, 0.1, 0.2])

frame_counter = Counter('frames_processed_total', 
                        'Total number of frames processed')

gesture_counter = Counter('gestures_detected_total', 
                          'Total gestures detected', 
                          ['gesture_type'])

cpu_gauge = Gauge('system_cpu_usage_percent', 
                  'Current CPU usage percentage')

memory_gauge = Gauge('system_memory_usage_percent', 
                     'Current memory usage percentage')

fps_gauge = Gauge('camera_fps', 
                  'Current frames per second')

In [5]:
# Data buffers for plotting
class MetricsBuffer:
    """Store recent metrics for real-time plotting"""
    def __init__(self, maxlen=100):
        self.latencies = deque(maxlen=maxlen)
        self.cpu_usage = deque(maxlen=maxlen)
        self.memory_usage = deque(maxlen=maxlen)
        self.fps_values = deque(maxlen=maxlen)
        self.timestamps = deque(maxlen=maxlen)
        self.gesture_counts = {}
        
    def add_latency(self, value):
        self.latencies.append(value)
        self.timestamps.append(datetime.now())
        
    def add_cpu(self, value):
        self.cpu_usage.append(value)
        
    def add_memory(self, value):
        self.memory_usage.append(value)
        
    def add_fps(self, value):
        self.fps_values.append(value)
        
    def increment_gesture(self, gesture):
        if gesture not in self.gesture_counts:
            self.gesture_counts[gesture] = 0
        self.gesture_counts[gesture] += 1
        
    def get_latest_metrics(self):
        return {
            'latency': list(self.latencies),
            'cpu': list(self.cpu_usage),
            'memory': list(self.memory_usage),
            'fps': list(self.fps_values),
            'timestamps': list(self.timestamps),
            'gestures': dict(self.gesture_counts)
        }

In [6]:
# Global metrics buffer
metrics_buffer = MetricsBuffer(maxlen=100)

# Start Prometheus HTTP server
try:
    start_http_server(8000)
    print("✅ Prometheus metrics server started on port 8000")
except OSError as e:
    if "Address already in use" in str(e):
        print("✅ Prometheus server already running on port 8000")
    else:
        print(f"⚠️ Could not start Prometheus server: {e}")

print("🔧 Metrics collection system initialized")

✅ Prometheus metrics server started on port 8000
🔧 Metrics collection system initialized


---
## 4. Plotting Functions for Metrics Visualization

In [7]:
def create_latency_plot():
    """Creates real time latency plot"""
    metrics = metrics_buffer.get_latest_metrics()
    
    if not metrics['latency']:
        # Returns empty plot if there's no data
        fig = go.Figure()
        fig.update_layout(
            title="Latency Over Time (ms)",
            xaxis_title="Frames",
            yaxis_title="Latency (ms)",
            template="plotly_dark",
            height=300
        )
        return fig
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=metrics['latency'],
        mode='lines+markers',
        name='Latency',
        line=dict(color='#00ff41', width=2),
        marker=dict(size=4)
    ))
    
    # Adds average line
    avg_latency = np.mean(metrics['latency'])
    fig.add_hline(y=avg_latency, line_dash="dash", 
                  line_color="yellow",
                  annotation_text=f"Avg: {avg_latency:.2f}ms")
    
    fig.update_layout(
        title=f"Latency Over Time - Avg: {avg_latency:.2f}ms",
        xaxis_title="Frame Number",
        yaxis_title="Latency (ms)",
        template="plotly_dark",
        height=300,
        showlegend=True
    )
    
    return fig

In [8]:
def create_cpu_memory_plot():
    """Creates real time CPU and Memory usage plot"""
    metrics = metrics_buffer.get_latest_metrics()
    
    fig = go.Figure()
    
    if metrics['cpu']:
        fig.add_trace(go.Scatter(
            y=metrics['cpu'],
            mode='lines',
            name='CPU Usage',
            line=dict(color='#ff6b6b', width=2),
            fill='tozeroy'
        ))
    
    if metrics['memory']:
        fig.add_trace(go.Scatter(
            y=metrics['memory'],
            mode='lines',
            name='Memory Usage',
            line=dict(color='#4ecdc4', width=2),
            fill='tozeroy'
        ))
    
    fig.update_layout(
        title="System Resource Usage",
        xaxis_title="Time",
        yaxis_title="Usage (%)",
        yaxis_range=[0, 100],
        template="plotly_dark",
        height=300,
        showlegend=True
    )
    
    return fig

In [9]:
def create_gesture_distribution_plot():
    """Creates hand gesture frequency bar chart"""
    metrics = metrics_buffer.get_latest_metrics()
    gestures = metrics['gestures']
    
    if not gestures:
        fig = go.Figure()
        fig.update_layout(
            title="Gesture Distribution",
            template="plotly_dark",
            height=300
        )
        return fig
    
    # Sort by frequency
    sorted_gestures = sorted(gestures.items(), key=lambda x: x[1], reverse=True)
    gesture_names = [g[0] for g in sorted_gestures]
    gesture_counts = [g[1] for g in sorted_gestures]
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=gesture_names,
        y=gesture_counts,
        marker=dict(
            color=gesture_counts,
            colorscale='Viridis',
            showscale=True
        ),
        text=gesture_counts,
        textposition='auto'
    ))
    
    fig.update_layout(
        title="Gesture Detection Frequency",
        xaxis_title="Gesture Type",
        yaxis_title="Count",
        template="plotly_dark",
        height=300
    )
    
    return fig

In [10]:
def create_fps_plot():
    """Creates FPS monitoring plot"""
    metrics = metrics_buffer.get_latest_metrics()
    
    if not metrics['fps']:
        fig = go.Figure()
        fig.update_layout(
            title="Frames Per Second (FPS)",
            template="plotly_dark",
            height=250
        )
        return fig
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=metrics['fps'],
        mode='lines',
        name='FPS',
        line=dict(color='#95e1d3', width=3),
        fill='tozeroy'
    ))
    
    avg_fps = np.mean(metrics['fps'])
    fig.add_hline(y=avg_fps, line_dash="dash", 
                  line_color="orange",
                  annotation_text=f"Avg: {avg_fps:.1f} FPS")
    
    fig.update_layout(
        title=f"Camera FPS - Avg: {avg_fps:.1f}",
        xaxis_title="Time",
        yaxis_title="FPS",
        template="plotly_dark",
        height=250
    )
    
    return fig

In [11]:
def get_metrics_summary():
    """Gets text summary of current metrics"""
    metrics = metrics_buffer.get_latest_metrics()
    
    summary = "📊 **Real-Time Metrics Summary**\n\n"
    
    if metrics['latency']:
        avg_latency = np.mean(metrics['latency'])
        min_latency = np.min(metrics['latency'])
        max_latency = np.max(metrics['latency'])
        summary += f"**Latency:**\n"
        summary += f"  - Average: {avg_latency:.2f} ms\n"
        summary += f"  - Min: {min_latency:.2f} ms\n"
        summary += f"  - Max: {max_latency:.2f} ms\n\n"
    
    if metrics['cpu']:
        avg_cpu = np.mean(metrics['cpu'])
        summary += f"**CPU Usage:** {avg_cpu:.1f}%\n"
    
    if metrics['memory']:
        avg_memory = np.mean(metrics['memory'])
        summary += f"**Memory Usage:** {avg_memory:.1f}%\n\n"
    
    if metrics['fps']:
        avg_fps = np.mean(metrics['fps'])
        summary += f"**FPS:** {avg_fps:.1f}\n\n"
    
    total_gestures = sum(metrics['gestures'].values())
    summary += f"**Total Gestures Detected:** {total_gestures}\n"
    
    return summary

---
## 5. Session State Management

In [ ]:
class SessionState:
    """Manages user session across Gradio interface"""
    def __init__(self):
        self.current_user = None
        self.is_logged_in = False
        self.detection_running = False
    
    def login(self, username):
        """Login our user"""
        self.current_user = username
        self.is_logged_in = True
    
    def logout(self):
        """Logout our user"""
        self.current_user = None
        self.is_logged_in = False
        self.detection_running = False
    
    def get_username(self):
        """Get current username"""
        return self.current_user if self.current_user else "Guest"

# Global session
session = SessionState()

---
## 6. Emergency Stop & Control Functions

In [13]:
stop_event = threading.Event()

def emergency_stop():
    """Functional emergency stop - actually stops the camera"""
    stop_event.set()  # Signal the stream to stop
    print("🚨 Emergency stop activated!")
    return "🚨 System stopped! Click 'Resume Detection' to restart."

def reset_and_start():
    """Reset stop flag before starting"""
    stop_event.clear()  # Reset the flag
    print("✅ System starting...")
    return "✅ System running..."

---
## 7. Load Pre-trained ML Model

In [14]:
# Loading the pre-trained SVM model
classification_model = joblib.load('./SVM_Models/best_label_classification_model.pkl')
classification_model

,steps,"[('svm', ...)]"
,transform_input,None
,memory,None
,verbose,False
,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False


In [15]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress INFO and WARNING messages

---
## 8. Arduino Serial Communication

In [16]:
# Communicating with our Arduino board
import serial
import time

arduino = serial.Serial('/dev/tty.usbserial-10', 9600)  # Arduino port
time.sleep(2) 

def send_label(label):
    message = label + '\n'
    arduino.write(message.encode())
    print(label)

---
## 9. MediaPipe Hand Detection Setup

In [17]:
# Mediapipe setup to detect the user's both hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.8, min_tracking_confidence=0.7)

I0000 00:00:1764251122.130552  390692 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M3 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


W0000 00:00:1764251122.136207  399289 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1764251122.140550  399289 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [18]:
def encode_fingers(hand_landmarks, hand_type):
    tips = [mp_hands.HandLandmark.THUMB_TIP,
            mp_hands.HandLandmark.INDEX_FINGER_TIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
            mp_hands.HandLandmark.RING_FINGER_TIP,
            mp_hands.HandLandmark.PINKY_TIP]
    dips = [mp_hands.HandLandmark.THUMB_IP,
            mp_hands.HandLandmark.INDEX_FINGER_DIP,
            mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
            mp_hands.HandLandmark.RING_FINGER_DIP,
            mp_hands.HandLandmark.PINKY_DIP]

    finger_status = []
    for i, (tip_id, dip_id) in enumerate(zip(tips, dips)):
        tip = hand_landmarks.landmark[tip_id]
        dip = hand_landmarks.landmark[dip_id]
        if i == 0:
            is_open = 1 if (tip.x < dip.x if hand_type == "Right" else tip.x > dip.x) else 0
        else:
            is_open = 1 if tip.y < dip.y else 0
        finger_status.append(is_open)
    return finger_status

---
## 10. Video Processing and Live Video Feed Functions

In [19]:
# Video processing function
def process_frame(frame):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    features = [0]*10
    if results.multi_hand_landmarks and results.multi_handedness:
        for hand_idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            hand_type = results.multi_handedness[hand_idx].classification[0].label
            fingers = encode_fingers(hand_landmarks, hand_type)
            if hand_type == "Left": features[:5] = fingers
            elif hand_type == "Right": features[5:] = fingers
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    return frame, features

In [20]:
def run_live_feed():
    """COllection of metrics and implementing stop signal"""
    cap = cv2.VideoCapture(0)
    latency_list = []
    frame_count = 0
    fps_start_time = time.time()
    fps_frame_count = 0
    
    stop_event.clear()  # Resets the flag when starting
    
    try:
        while not stop_event.is_set():
            ret, frame = cap.read()
            if not ret:
                print("⚠️ Failed to read frame")
                break
            
            # Starts timing for latency
            start = time.time()
            frame = cv2.flip(frame, 1)
            frame, features = process_frame(frame)
            
            # Prediction
            if any(features):
                input_df = pd.DataFrame([features], columns=classification_model.feature_names_in_)
                prediction = classification_model.predict(input_df)[0]
            else:
                prediction = "No Hand"
            
            # Sends to Arduino
            try:
                send_label(prediction)
            except Exception as e:
                print(f"⚠️ Arduino error: {e}")
            
            # Calculates latency
            latency = (time.time() - start) * 1000  # Convert to ms
            latency_list.append(latency)
            
            # Updating Prometheus metrics
            latency_histogram.observe(latency / 1000)  # Prometheus expects seconds
            frame_counter.inc()
            
            if prediction != "No Hand" and prediction != "invalid":
                gesture_counter.labels(gesture_type=prediction).inc()
                metrics_buffer.increment_gesture(prediction)
            
            # Updates metrics buffer for plotting
            metrics_buffer.add_latency(latency)
            
            # Updates CPU and Memory every 10 frames (to avoid overhead)
            if frame_count % 10 == 0:
                cpu_percent = psutil.cpu_percent(interval=0)
                memory_percent = psutil.virtual_memory().percent
                
                cpu_gauge.set(cpu_percent)
                memory_gauge.set(memory_percent)
                
                metrics_buffer.add_cpu(cpu_percent)
                metrics_buffer.add_memory(memory_percent)
            
            # Calculates FPS every second
            fps_frame_count += 1
            if time.time() - fps_start_time >= 1.0:
                current_fps = fps_frame_count / (time.time() - fps_start_time)
                fps_gauge.set(current_fps)
                metrics_buffer.add_fps(current_fps)
                
                fps_frame_count = 0
                fps_start_time = time.time()
            
            # Displays info on frame
            cv2.putText(frame, f"Gesture: {prediction}", (10, 40), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.putText(frame, f"Latency: {latency:.1f} ms", (10, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            # Add CPU and FPS overlay
            if frame_count % 10 == 0:
                cv2.putText(frame, f"CPU: {cpu_percent:.1f}%", (10, 120), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            
            frame_count += 1
            yield frame  # Sends frame to Gradio
            
    finally:
        cap.release()
        cv2.destroyAllWindows()
        avg_latency = np.mean(latency_list) if latency_list else 0
        print(f"📊 Camera released. Avg Latency: {avg_latency:.2f} ms")

---
## 11. User Manager Initialization

In [21]:
# Initializing User Manager
user_manager = get_user_manager()
print("✅ User manager initialized!!!")

📂 User database found: users_database.csv
✅ User manager initialized!!!


---
## 12. Gradio User Interface

In [ ]:
# Plotting Update functions for Gradio
def update_plots():
    """Updates all plots periodically"""
    latency_fig = create_latency_plot()
    cpu_memory_fig = create_cpu_memory_plot()
    gesture_fig = create_gesture_distribution_plot()
    fps_fig = create_fps_plot()
    summary_text = get_metrics_summary()
    
    return latency_fig, cpu_memory_fig, gesture_fig, fps_fig, summary_text


def reset_metrics():
    """Resets all metrics buffers"""
    global metrics_buffer
    metrics_buffer = MetricsBuffer(maxlen=100)
    return "✅ Metrics reset successfully!"

In [24]:
# Custom CSS for Gradio
custom_css = """
#main-title {
    text-align: center;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 30px;
    border-radius: 10px;
    margin-bottom: 20px;
}

#login-box {
    max-width: 500px;
    margin: 0 auto;
    padding: 30px;
    background: white;
    border-radius: 15px;
    box-shadow: 0 10px 30px rgba(0,0,0,0.1);
}

#camera-feed {
    border: 3px solid #667eea;
    border-radius: 10px;
    box-shadow: 0 5px 15px rgba(0,0,0,0.2);
}

#gesture-info {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    border-radius: 10px;
    text-align: center;
    font-size: 24px;
    font-weight: bold;
}

.control-button {
    font-size: 18px !important;
    padding: 15px 30px !important;
    border-radius: 10px !important;
}
"""

In [ ]:
# Page 1 functions
def handle_register(username, password, confirm_password):
    """Handle user registration"""
    if password != confirm_password:
        return "❌ Passwords do not match!", gr.update(visible=True), gr.update(visible=False)
    
    success, message = user_manager.register_user(username, password)
    return message, gr.update(visible=True), gr.update(visible=False)


def handle_login(username, password):
    """Handle user login"""
    success, message = user_manager.login_user(username, password)
    
    if success:
        session.login(username)
        return (
            message,
            gr.update(visible=False),  # Hides login page
            gr.update(visible=True),   # Shows detection page
            gr.update(visible=False),  # Hides feedback page
            gr.update(value=f"👤 User: {username}")  # Updates user display
        )
    else:
        return (
            message,
            gr.update(visible=True),   # Keeps login page visible
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(value="")
        )


In [26]:
# Page 2 Functions
def start_detection_system():
    """Starts the detection system"""
    session.detection_running = True
    stop_event.clear()
    return (
        "🎥 Detection started! Show your gestures to the camera.",
        gr.update(visible=False, interactive=False),  # Start button is hidden
        gr.update(visible=True, interactive=True),   # Pause button is visible
        gr.update(visible=True, interactive=True),   # Quit button is visible
        gr.update(visible=True, interactive=True)    # Emergency button is visible
    )


def pause_detection():
    """Pause detection"""
    stop_event.set()
    return (
        "⏸️ Detection paused. Click 'Resume' to continue.",
        gr.update(value="▶️ Resume", interactive=True)
    )


def resume_detection():
    """Resume detection"""
    stop_event.clear()
    return (
        "▶️ Detection resumed!",
        gr.update(value="⏸️ Pause", interactive=True)
    )


def quit_detection():
    """Quits detection and goes to feedback page"""
    stop_event.set()
    session.detection_running = False
    
    return (
        gr.update(visible=False),  # Hides detection page
        gr.update(visible=True),   # Shows feedback page
        f"Thank you for using VisionARM, {session.get_username()}! 🎉"
    )


def emergency_stop_and_exit():
    """Emergency stop and go to feedback"""
    stop_event.set()
    session.detection_running = False
    
    return (
        gr.update(visible=False),  # Hides detection page
        gr.update(visible=True),   # Shows feedback page
        f"🚨 Emergency stop activated! System safely stopped for {session.get_username()}."
    )

In [28]:
# Page 3 Functions
def submit_feedback_and_logout(feedback_text):
    """Submit feedback and logout"""
    username = session.get_username()
    
    if feedback_text.strip():
        success, message = user_manager.save_feedback(username, feedback_text)
        feedback_msg = message
    else:
        feedback_msg = "No feedback provided."
    
    # Logout our user
    session.logout()
    
    return (
        f"{feedback_msg}\n\n🔓 You have been logged out. Thank you for using VisionARM!",
        gr.update(visible=True),   # Shows login page
        gr.update(visible=False),  # Hides detection page
        gr.update(visible=False)   # Hides feedback page
    )


def skip_feedback_and_logout():
    """Skips feedback and logout"""
    username = session.get_username()
    session.logout()
    
    return (
        f"👋 Goodbye, {username}! You have been logged out.",
        gr.update(visible=True),
        gr.update(visible=False),
        gr.update(visible=False)
    )


In [29]:
with gr.Blocks(css=custom_css, title="VisionARM - Gesture Control System") as demo:
    
    # hidden state variables
    current_page = gr.State("login")
    
    # ++++++++++ PAGE 1: LOGIN / REGISTER ++++++++++
    with gr.Column(visible=True) as login_page:
        gr.Markdown("""
        <div id="main-title">
            <h1>🖐️ VisionARM</h1>
            <h3>Hand Gesture Controlled Robotic Arm</h3>
            <p>University of Florida | Artificial Intelligence Systems Project</p>
        </div>
        """)
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("## 🔐 Login")
                login_username = gr.Textbox(
                    label="Username",
                    placeholder="Enter your username",
                    elem_id="login-box"
                )
                login_password = gr.Textbox(
                    label="Password",
                    placeholder="Enter your password",
                    type="password",
                    elem_id="login-box"
                )
                login_btn = gr.Button("🔓 Login", variant="primary", size="lg", elem_classes="control-button")
                login_status = gr.Textbox(label="Status", interactive=False)
            
            with gr.Column():
                gr.Markdown("## 📝 Register")
                register_username = gr.Textbox(
                    label="Username",
                    placeholder="Choose a username (min 3 characters)",
                    elem_id="login-box"
                )
                register_password = gr.Textbox(
                    label="Password",
                    placeholder="Choose a password (min 6 characters)",
                    type="password",
                    elem_id="login-box"
                )
                register_confirm_password = gr.Textbox(
                    label="Confirm Password",
                    placeholder="Re-enter your password",
                    type="password",
                    elem_id="login-box"
                )
                register_btn = gr.Button("📝 Register", variant="secondary", size="lg", elem_classes="control-button")
                register_status = gr.Textbox(label="Status", interactive=False)
        
        gr.Markdown("""
        ---
        ### 📋 Instructions:
        - **New user?** Register with a unique username and password (minimum 6 characters)
        - **Existing user?** Login with your credentials
        - Your data is securely stored and encrypted
        """)
    
    # ++++++++++ PAGE 2: DETECTION ++++++++++
    with gr.Column(visible=False) as detection_page:
        gr.Markdown("""
        <div id="main-title">
            <h1>🎥 Live Detection</h1>
            <h3>Real-time Hand Gesture Recognition</h3>
        </div>
        """)
        
        # User info display
        user_display = gr.Textbox(
            label="Current User",
            value="",
            interactive=False,
            elem_id="gesture-info"
        )
        
        with gr.Row():
            # Camera feed
            with gr.Column(scale=1):
                camera_feed = gr.Image(
                    label="Camera Feed",
                    streaming=True,
                    height=480,
                    elem_id="camera-feed"
                )
                
                detection_status = gr.Textbox(
                    label="System Status",
                    value="Ready to start",
                    interactive=False
                )
                
                # Control buttons
                with gr.Row():
                    start_btn = gr.Button(
                        "▶️ Start Detection",
                        variant="primary",
                        size="lg",
                        elem_classes="control-button"
                    )
                    pause_resume_btn = gr.Button(
                        "⏸️ Pause",
                        variant="secondary",
                        size="lg",
                        visible=False,
                        elem_classes="control-button"
                    )
                    quit_btn = gr.Button(
                        "🚪 Quit",
                        variant="secondary",
                        size="lg",
                        visible=False,
                        elem_classes="control-button"
                    )
                    emergency_btn = gr.Button(
                        "🚨 EMERGENCY STOP",
                        variant="stop",
                        size="lg",
                        visible=False,
                        elem_classes="control-button"
                    )
            
            # Performance Metrics 
            with gr.Column(scale=1):
                gr.Markdown("### 📊 Performance Metrics")
                
                metrics_summary = gr.Markdown("Waiting for data...")
                latency_plot = gr.Plot(label="Latency Monitoring")
                cpu_memory_plot = gr.Plot(label="System Resources")
                fps_plot = gr.Plot(label="Frame Rate")
                gesture_plot = gr.Plot(label="Gesture Distribution")
                
                with gr.Row():
                    refresh_plots_btn = gr.Button("🔄 Refresh Plots", variant="secondary")
                    reset_metrics_btn = gr.Button("🗑️ Reset Metrics", variant="secondary")
        
        gr.Markdown("""
        ---
        **Tips:**
        - Keep your hands clearly visible to the camera
        - Ensure good lighting for best results
        - Maintain a distance of 1-2 feet from camera
        - Use **Emergency Stop** if robot behaves unexpectedly
        """)
        
        # Auto refresh timer for plots
        refresh_timer = gr.Timer(value=2, active=True)
        refresh_timer.tick(
            fn=update_plots,
            outputs=[latency_plot, cpu_memory_plot, gesture_plot, fps_plot, metrics_summary]
        )
    
    # ++++++++++ PAGE 3: FEEDBACK / THANK YOU ++++++++++
    with gr.Column(visible=False) as feedback_page:
        gr.Markdown("""
        <div id="main-title">
            <h1>🎉 Thank You!</h1>
            <h3>Your session has ended</h3>
        </div>
        """)
        
        feedback_message = gr.Textbox(
            label="Session Summary",
            value="",
            interactive=False,
            elem_id="gesture-info"
        )
        
        gr.Markdown("### 💬 Share Your Feedback (Optional)")
        feedback_text = gr.Textbox(
            label="How was your experience?",
            placeholder="Tell us what you think about VisionARM...",
            lines=5
        )
        
        with gr.Row():
            submit_feedback_btn = gr.Button("📤 Submit Feedback & Logout", variant="primary", size="lg")
            skip_feedback_btn = gr.Button("⏭️ Skip & Logout", variant="secondary", size="lg")
        
        feedback_result = gr.Textbox(label="Status", interactive=False)
    
    # ++++++++++ EVENT HANDLERS ++++++++++
    
    # Registration
    register_btn.click(
        fn=handle_register,
        inputs=[register_username, register_password, register_confirm_password],
        outputs=[register_status, login_page, detection_page]
    )
    
    # Login
    login_btn.click(
        fn=handle_login,
        inputs=[login_username, login_password],
        outputs=[login_status, login_page, detection_page, feedback_page, user_display]
    )
    
    # Start Detection
    start_btn.click(
        fn=start_detection_system,
        outputs=[detection_status, start_btn, pause_resume_btn, quit_btn, emergency_btn]
    ).then(
        fn=run_live_feed,
        outputs=camera_feed
    )
    
    # Pause/Resume
    pause_resume_btn.click(
        fn=lambda: pause_detection() if stop_event.is_set() else resume_detection(),
        outputs=[detection_status, pause_resume_btn]
    )
    
    # Quit
    quit_btn.click(
        fn=quit_detection,
        outputs=[detection_page, feedback_page, feedback_message]
    )
    
    # Emergency Stop
    emergency_btn.click(
        fn=emergency_stop_and_exit,
        outputs=[detection_page, feedback_page, feedback_message]
    )
    
    # Refresh Plots
    refresh_plots_btn.click(
        fn=update_plots,
        outputs=[latency_plot, cpu_memory_plot, gesture_plot, fps_plot, metrics_summary]
    )
    
    # Reset Metrics
    reset_metrics_btn.click(
        fn=reset_metrics,
        outputs=detection_status
    )
    
    # Feedback Submission
    submit_feedback_btn.click(
        fn=submit_feedback_and_logout,
        inputs=feedback_text,
        outputs=[feedback_result, login_page, detection_page, feedback_page]
    )
    
    # Skip Feedback
    skip_feedback_btn.click(
        fn=skip_feedback_and_logout,
        outputs=[feedback_result, login_page, detection_page, feedback_page]
    )

# Launch message
print("🚀 Starting Gradio interface...")

🚀 Starting Gradio interface...


---
## 13. Lauching our Application

In [30]:
if __name__ == "__main__":
    # Get user statistics
    stats = user_manager.get_user_stats()
    
    print("\n" + "="*60)
    print("📊 DATABASE STATISTICS:")
    print(f"  Total registered users: {stats['total_users']}")
    print(f"  Users who provided feedback: {stats['users_with_feedback']}")
    print("="*60)
    
    print("\n🌐 Starting Gradio interface...")
    print("🔐 First time? Register a new account!")
    print("="*60 + "\n")
    
    demo.launch(
        share=False,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
    )


📊 DATABASE STATISTICS:
  Total registered users: 2
  Users who provided feedback: 2

🌐 Starting Gradio interface...
🔐 First time? Register a new account!

* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.


No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand


W0000 00:00:1764261911.092109  399290 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
No Hand
invalid
grab
No Hand
No Hand
No Hand
No Hand
No Hand
invalid
drop
drop
drop
drop
drop
drop
invalid
grab
grab
grab
grab
grab
grab
grab
grab
grab
up
up
up
up
down
down
down
down
down
down
down
down
down
down
left
left
left
left
left
left
left
left
left
left
left
left
left
left
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
right
up
up
up
up
up
up
up
down
down
down
down
down
down
down
down
down
down
down
up
up
up
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
invalid
grab
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
up
down
down
down
down
down
down
down
drop
drop
drop
drop